# Lab 10 - Agent

In [198]:
require 'json'
require 'sqlite3'
require 'time'
require 'openai'
require 'narray'
require 'dotenv'
require 'date'

# Load environment variables from .env file
Dotenv.load

# Initialize OpenAI client using environment variable
$client = OpenAI::Client.new(access_token: ENV['OPENAI_API_KEY'])

#<OpenAI::Client:49440 @api_type=nil, @api_version="v1", @access_token=[REDACTED], @log_errors=false, @organization_id=[REDACTED], @uri_base="https://api.openai.com/", @request_timeout=120, @extra_headers=[REDACTED], @faraday_middleware=nil>

## Our database schema, using SQLite

In [199]:
require 'fileutils'
require 'sqlite3'

db_filename = 'event_planning.db'

# Try to delete the file if it exists
begin
  FileUtils.rm(db_filename) if File.exist?(db_filename)
  puts "File '#{db_filename}' deleted successfully."
rescue StandardError => e
  # Silently handle any errors, similar to the Python version
end

# Connect to SQLite
# db = SQLite3::Database.new(db_filename)
# db.results_as_hash = true  # This makes query results return as hashes instead of arrays

$db = SQLite3::Database.new('event_planning.db')
$db.results_as_hash = true



# Function to initialize the database and create tables
def initialize_database(db)
  # Create 'events' table
  $db.execute(<<-SQL)
    CREATE TABLE IF NOT EXISTS events (
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      location TEXT NOT NULL,
      date TEXT NOT NULL
    )
  SQL

  # Create 'attendees' table
  $db.execute(<<-SQL)
    CREATE TABLE IF NOT EXISTS attendees (
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      event_id INTEGER NOT NULL,
      full_name TEXT NOT NULL,
      FOREIGN KEY (event_id) REFERENCES events (id)
    )
  SQL
end

# Initialize the database and create the tables
initialize_database(db)

File 'event_planning.db' deleted successfully.


[]

## Calculates the cost of an OpenAI request

In [200]:
def cost(response)
  # Define prices per million tokens for each model version
  prices_per_million = {
    "gpt-4o" => { "input" => 5.00, "output" => 15.00 },
    "gpt-4o-2024-08-06" => { "input" => 2.50, "output" => 10.00 },
    "gpt-4o-2024-05-13" => { "input" => 5.00, "output" => 15.00 },
    "gpt-4o-mini" => { "input" => 0.15, "output" => 0.60 },
    "gpt-4o-mini-2024-07-18" => { "input" => 0.15, "output" => 0.60 },
    "gpt-3.5-turbo" => { "input" => 0.003, "output" => 0.006 },
    "davinci-002" => { "input" => 12.00, "output" => 12.00 },
    "babbage-002" => { "input" => 1.60, "output" => 1.60 },
    "text-embedding-3-small" => { "input" => 0.020, "output" => 0.020 },
    "text-embedding-3-large" => { "input" => 0.130, "output" => 0.130 },
    "ada-v2" => { "input" => 0.100, "output" => 0.100 }
  }

  model_version = response.model
  
  if prices_per_million.key?(model_version)
    input_price_per_million = prices_per_million[model_version]["input"]
    output_price_per_million = prices_per_million[model_version]["output"]
  else
    raise "Pricing information for model '#{model_version}' is not available."
  end

  # Get token usage
  prompt_tokens = response.usage.prompt_tokens
  completion_tokens = response.usage.completion_tokens

  # Calculate costs in dollars
  input_cost = (prompt_tokens.to_f / 1_000_000) * input_price_per_million
  output_cost = (completion_tokens.to_f / 1_000_000) * output_price_per_million
  total_cost = input_cost + output_cost

  total_cost
end

:cost

## Helpers to process structured OpenAI responses

In [201]:
def tool_response(response)
  begin
    message = response.dig("choices", 0, "message")
    return nil unless message && message["tool_calls"]
    
    tool_call = message["tool_calls"][0]
    return nil unless tool_call && tool_call["function"]
    
    arguments_str = tool_call["function"]["arguments"]
    JSON.parse(arguments_str)
  rescue StandardError => e
    puts "Error parsing tool response: #{e}"
    nil
  end
end

def chain_of_thought_response(response)
  begin
    content = response.dig("choices", 0, "message", "content")
    return nil unless content
    
    content_json = JSON.parse(content)
    content_json["final_answer"]
  rescue StandardError => e
    puts "Error parsing chain of thought response: #{e}"
    nil
  end
end



:chain_of_thought_response

In [202]:
# def tool_response(response)
#   begin
#     content = response.choices[0]
#     tool_call = content.message.tool_calls[0]
#     arguments_str = tool_call.function.arguments
#     JSON.parse(arguments_str)
#   rescue StandardError => e
#     puts response
#     puts e
#     nil
#   end
# end

In [203]:
# def chain_of_thought_response(response)
#   begin
#     choice = response.choices[0]
#     content = choice.message.content
#     content_json = JSON.parse(content)
#     content_json["final_answer"]
#   rescue StandardError => e
#     puts response
#     puts e
#     nil
#   end
# end

## OpenAI completion request methods

In [204]:
# def complete_chain_of_thought(prompt, function)
#   # Extract function components for the template
#   name = function["name"]
#   description = function["description"]
#   parameters = function["parameters"]
  
#   resp = $client.chat(
#     parameters: {
#       messages: [
#         {
#           role: "system",
#           content: "You are a helpful event planning assistant. Always analyze and think step-by-step before responding with the final answer."
#         },
#         {
#           role: "user",
#           content: prompt
#         }
#       ],
#       model: "gpt-4o-mini",
#       temperature: 0,
#       response_format: {
#         type: "json_schema",
#         json_schema: {
#           name: name,
#           description: description,
#           strict: true,
#           schema: {
#             type: "object",
#             properties: {
#               steps: {
#                 type: "array",
#                 items: {
#                   type: "object",
#                   properties: {
#                     explanation: {
#                       type: "string"
#                     },
#                     output: {
#                       type: "string"
#                     }
#                   },
#                   required: ["explanation", "output"],
#                   additional_properties: false
#                 }
#               },
#               final_answer: parameters
#             },
#             required: ["steps", "final_answer"],
#             additional_properties: false
#           }
#         }
#       }
#     }
#   )
  
#   chain_of_thought_response(resp)
# end

def complete_chain_of_thought(prompt, function)
  response = $client.chat(
    parameters: {
      model: "gpt-4",
      messages: [
        {
          role: "system",
          content: "You are a helpful event planning assistant. Always analyze and think step-by-step before responding with the final answer."
        },
        {
          role: "user",
          content: prompt
        }
      ],
      temperature: 0,
      response_format: { 
        type: "json_object"
      },
      tools: [{  # Add tools configuration
        type: "function",
        function: function
      }],
      tool_choice: "auto"
    }
  )
  chain_of_thought_response(response)
end

:complete_chain_of_thought

In [205]:
# def complete_function(prompt, function, chain = false)
#   # If chain is true, use the chain of thought completion
#   return complete_chain_of_thought(prompt, function) if chain
  
#   # Otherwise, proceed with normal function calling
#   resp = $client.chat(
#     parameters: {
#       messages: [
#         {
#           role: "system",
#           content: "You are a helpful event planning assistant."
#         },
#         {
#           role: "user",
#           content: prompt
#         }
#       ],
#       model: "gpt-4o",
#       temperature: 0,
#       tools: [
#         {
#           type: "function",
#           function: function
#         }
#       ]
#     }
#   )
  
#   tool_response(resp)
# end

# def complete_function(prompt, function, chain: false)
#   if chain
#     return complete_chain_of_thought(prompt, function)
#   end
  
#   response = $client.chat(
#     parameters: {
#       model: "gpt-4",
#       messages: [
#         {
#           role: "system",
#           content: "You are a helpful event planning assistant."
#         },
#         {
#           role: "user",
#           content: prompt
#         }
#       ],
#       tools: [{
#         type: "function",
#         function: function
#       }],
#       tool_choice: "auto"
#     }
#   )
#   tool_response(response)
# end

def complete_chain_of_thought(prompt, function)
  response = $client.chat(
    parameters: {
      model: "gpt-4",
      messages: [
        {
          role: "system",
          content: "You are a helpful event planning assistant. Always analyze and think step-by-step before responding with the final answer."
        },
        {
          role: "user",
          content: prompt
        }
      ],
      temperature: 0,
      function_call: { name: function["name"] },
      functions: [function]  # Changed from tools to functions
    }
  )
  chain_of_thought_response(response)
end

def get_list_events_function
  {
    "name" => "events_query",
    "description" => "Filter parameters to query an events SQL table based on dates.",
    "parameters" => {
      "type" => "object",
      "properties" => {
        "after_date" => {
          "type" => "string",
          "description" => "The date formatted in YYYY-MM-DD"
        },
        "before_date" => {
          "type" => "string",
          "description" => "The date formatted in YYYY-MM-DD"
        }
      },
      "required" => ["after_date", "before_date"],
      "additionalProperties" => false
    }
  }
end

:get_list_events_function

## Task Classification

In [206]:
def get_classification_prompt(query)
  <<~PROMPT
    # Instructions

    For an event management SaaS product, natural language queries need to be classified and routed to the appropriate branch. Classify the following user query:
    
    #{query}
  PROMPT
end

:get_classification_prompt

In [207]:
def get_classification_function(methods_enum)
  {
    "name" => "task_type",
    "description" => "Classify the task based on the user query.",
    "parameters" => {
      "type" => "object",
      "properties" => {
        "task_type" => {
          "type" => "string",
          "enum" => methods_enum
        }
      },
      "required" => ["task_type"],
      "additionalProperties" => false  # Changed from additional_properties
    }
  }
end

:get_classification_function

In [208]:
def classify(methods_enum, query)
  prompt = get_classification_prompt(query)
  function = get_classification_function(methods_enum)
  classification = complete_function(prompt, function)
  
  if classification
    task = classification['task_type']
    puts "#{query} => #{task}"
    task
  else
    nil
  end
end

:classify

In [209]:
def date_from_string(datestr)
  begin
    Date.strptime(datestr, '%Y-%m-%d')
  rescue Date::Error
    nil
  end
end

:date_from_string

## Create Event

In [210]:
def get_create_event_prompt(query)
  current_date = Time.now.strftime('%B %d, %Y')
  <<~PROMPT
    # Instructions

    Today's date is #{current_date}. For an event management SaaS product, extract the location and the date from the following user query. For dates that don't specify a year, always choose a date in the future:

    ## User Query

    #{query}
  PROMPT
end

def get_create_event_function
  {
    "name" => "event_details",
    "description" => "Extract the location and date for an event.",
    "strict" => true,
    "parameters" => {
      "type" => "object",
      "properties" => {
        "location" => {
          "type" => "string"
        },
        "date" => {
          "type" => "string",
          "description" => "The date formatted in YYYY-MM-DD"
        }
      },
      "required" => ["location", "date"],
      "additional_properties" => false
    }
  }
end

def create_event_sql(location, date)
  begin
    $db.execute(
      'INSERT INTO events (location, date) VALUES (?, ?)',
      [location, date]
    )
    puts "Event '#{location}' created successfully."
  rescue SQLite3::Exception => e
    puts "Error creating event: #{e}"
  end
end

def create_event(query, chain = false)
  args = complete_function(
    get_create_event_prompt(query),
    get_create_event_function,
    chain: chain
  )
  
  puts args
  if args
    location = args["location"]
    date = date_from_string(args["date"])
    create_event_sql(location, date)
  end
end

:create_event

In [211]:
#create_event("Book the Ellison Lodge A for December 4th", chain: true)

## List Events

In [212]:
def get_list_events_prompt(query)
  current_date = Time.now.strftime('%B %d, %Y')
  <<~PROMPT
    # Instructions

    Today's date is #{current_date}. For an event management SaaS product, a user is asking to list upcoming events for which an optional date range can be provided. For dates that don't specify a year, always choose a date in the future. Extract the date range from the following user query:
    
    #{query}
  PROMPT
end

# def get_list_events_function
#   {
#     "name" => "events_query",
#     "description" => "Filter parameters to query an events SQL table based on dates.",
#     "strict" => true,
#     "parameters" => {
#       "type" => "object",
#       "properties" => {
#         "after_date" => {
#           "type" => "string",
#           "description" => "The date formatted in YYYY-MM-DD"
#         },
#         "before_date" => {
#           "type" => "string",
#           "description" => "The date formatted in YYYY-MM-DD"
#         }
#       },
#       "required" => ["after_date", "before_date"],
#       "additional_properties" => false
#     }
#   }
# end

def list_events_sql(after_date: nil, before_date: nil, output: true)
  query = 'SELECT id, location, date FROM events'
  params = []
  
  # Add conditions for filtering by date
  if after_date && before_date
    query += ' WHERE date >= ? AND date <= ?'
    params.push(after_date, before_date)
  elsif after_date
    query += ' WHERE date >= ?'
    params.push(after_date)
  elsif before_date
    query += ' WHERE date <= ?'
    params.push(before_date)
  end
  
  # Execute the query with filters if any
  events = $db.execute(query, params)
  
  if events.any?
    if output
      events.each do |ev|
        puts "ID: #{ev[0]}, Location: #{ev[1]}, Date: #{ev[2]}"
      end
    end
    events
  else
    puts "No events found for the specified date range."
    nil
  end
end

def list_events(query, chain: false)
  args = complete_function(
    get_list_events_prompt(query),
    get_list_events_function,
    chain: chain
  )
  
  puts args
  if args
    after_date = date_from_string(args["after_date"])
    before_date = date_from_string(args["before_date"])
    list_events_sql(after_date: after_date, before_date: before_date)
  end
end

:list_events

In [213]:
list_events("highland", chain: true)

## Create Attendee

In [214]:
def get_events_enum
  events = list_events_sql(output: false)
  events_enum = events.map { |e| "#{e[1]} on #{e[2]}" }
  events_enum.push("other")
  events_enum
end

def get_event_id(event_str)
  return nil if event_str == "other"  # Fallback spotted! No event found
  
  events = list_events_sql(output: false)
  events.each do |ev|
    curr_event_str = "#{ev[1]} on #{ev[2]}"
    return ev[0] if event_str == curr_event_str  # Found the event!
  end
  
  nil
end

:get_event_id

In [215]:
def get_create_attendee_prompt(query)
  current_date = Time.now.strftime('%B %d, %Y')
  <<~PROMPT
    # Instructions

    Today's date is #{current_date}. For an event management SaaS product, extract the event and the attendee name from the following user query:
    
    #{query}
  PROMPT
end

def get_create_attendee_function(events_enum)
  {
    "name" => "attendee_rsvp_details",
    "description" => "Extract the event details and the attendee names.",
    "strict" => true,
    "parameters" => {
      "type" => "object",
      "properties" => {
        "event" => {
          "type" => "string",
          "enum" => events_enum
        },
        "attendees" => {
          "type" => "array",
          "items" => {
            "type" => "string"
          }
        }
      },
      "required" => ["event", "attendees"],
      "additional_properties" => false
    }
  }
end

def create_attendee_sql(event_id, full_name)
  begin
    $db.execute(
      'INSERT INTO attendees (event_id, full_name) VALUES (?, ?)',
      [event_id, full_name]
    )
    puts "Attendee '#{full_name}' added to event ID #{event_id}."
  rescue SQLite3::Exception => e
    puts "Error adding attendee: #{e}"
  end
end

def create_attendee(query, chain: false)
  events_enum = get_events_enum
  
  if events_enum&.any?
    args = complete_function(
      get_create_attendee_prompt(query),
      get_create_attendee_function(events_enum),
      chain: chain
    )
    
    puts args
    if args
      event_id = get_event_id(args["event"])
      if event_id
        names = args["attendees"]
        names.each do |name|
          create_attendee_sql(event_id, name)
        end
      end
    end
  else
    puts "[create_attendee] No events found! Please create an event first"
    nil
  end
end

:create_attendee

## List Attendees

In [216]:
def get_list_attendees_prompt(query)
  current_date = Time.now.strftime('%B %d, %Y')
  <<~PROMPT
    # Instructions

    Today's date is #{current_date}. For an event management SaaS product, extract the event from the following user query:
    
    #{query}
  PROMPT
end

def get_list_attendees_function(events_enum)
  {
    "name" => "event_query",
    "description" => "Identify the event details",
    "strict" => true,
    "parameters" => {
      "type" => "object",
      "properties" => {
        "event" => {
          "type" => "string",
          "enum" => events_enum
        }
      },
      "required" => ["event"],
      "additional_properties" => false
    }
  }
end

def list_attendees_sql(event_id)
  attendees = $db.execute(
    'SELECT id, full_name FROM attendees WHERE event_id = ?',
    [event_id]
  )
  
  if attendees.any?
    attendees.each do |attendee|
      puts "ID: #{attendee[0]}, Name: #{attendee[1]}"
    end
    attendees
  else
    puts "No attendees found for event ID #{event_id}."
    nil
  end
end

def list_attendees(query, chain: false)
  events_enum = get_events_enum
  
  if events_enum&.any?
    args = complete_function(
      get_list_attendees_prompt(query),
      get_list_attendees_function(events_enum),
      chain: chain
    )
    
    if args
      event_id = get_event_id(args["event"])
      list_attendees_sql(event_id)
    end
  else
    puts "[list_attendees] No events found! Please create an event first"
    nil
  end
end

:list_attendees

# Main Agent Loop

1. Accept a user query
2. Classify the query task (create event, list events, etc..)
3. Call appropriate task method
4. Repeat

In [217]:
def fallback(query, chain: false)
  puts "Fallback! #{query}"
end

def run(query, chain: true)
  methods = {
    "create_event" => method(:create_event),
    "list_events" => method(:list_events),
    "create_attendee" => method(:create_attendee),
    "list_attendees" => method(:list_attendees),
    "other" => method(:fallback)
  }
  
  puts '--------'
  methods_enum = methods.keys
  method_key = classify(methods_enum, query)
  method = methods[method_key]
  method.call(query, chain: chain)
end

:run

In [218]:
run("Book the Ellison Lodge A for December 4th")
run("Jane Doe is coming to the party on 12/4")
run("Pencil in highland park gazebo for june 11th")
run("Leah, Alice, and Fred are all coming to ellison in december")
run("Who is coming in december?")
run("What events are coming up?")

--------
Book the Ellison Lodge A for December 4th => create_event


Faraday::BadRequestError: the server responded with status 400

In [219]:
run("Steve's coming to highland park!")

--------
Steve's coming to highland park! => create_attendee
No events found for the specified date range.


NoMethodError: undefined method `map' for nil:NilClass

In [ ]:
run("highland list")

--------
highland list => other
Fallback! highland list


In [ ]:
run("highland attendees")

--------
highland attendees => list_attendees
No events found for the specified date range.


NoMethodError: undefined method `map' for nil:NilClass

## A (very) basic agent text input UI

In [ ]:
while True:
    user_input = input("Enter a string (type 'exit' to quit): ")
    if user_input.lower() == "exit":
        break
    run(user_input)

SyntaxError: (irb): syntax error, unexpected ':', expecting `do' for condition or ';' or '\n'
while True:
          ^
(irb):2: syntax error, unexpected ':', expecting `then' or ';' or '\n'
...f user_input.lower() == "exit":
...                              ^
